In [2]:
import os
import json
import math
import random
import glob
import numpy as np
import torch
import numpy.linalg as LA


import skimage.transform
import skimage.io

from scipy import io
import scipy.spatial.distance as scipy_spatial_dist

/home/kmuvcl/anaconda3/envs/debug/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def cos_cdis(x, y, semi_sphere=False):
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html#scipy.spatial.distance.cdist
    ### compute cosine distance
    ### scipy: same 0, opposite 2, orthorgonal 1, dist = 1-AB/(|A||B|)
    dist_cos = scipy_spatial_dist.cdist(x, y, 'cosine')  
    dist_cos *= -1.0
    dist_cos += 1.0

    if semi_sphere is True: dist_cos = np.abs(dist_cos)  # dist = abs|AB/(|A||B|)|
    dist_cos_arc = np.arccos(dist_cos)
    return dist_cos_arc
def orth(v):
    x, y, z = v
    o = np.array([0.0, -z, y] if abs(x) < abs(y) else [-z, 0.0, x])
    o /= LA.norm(o)
    return o

def gold_spiral_sampling_patch(v, alpha, num_pts):
    v1 = orth(v)
    v2 = np.cross(v, v1)
    v, v1, v2 = v[:, None], v1[:, None], v2[:, None]
    # indices = np.arange(num_pts) + 0.66
    indices = np.arange(num_pts) + 0.5
    phi = np.arccos(1 + (math.cos(alpha) - 1) * indices / num_pts)
    theta = np.pi * (1 + 5 ** 0.5) * indices
    r = np.sin(phi)
    return (v * np.cos(phi) + r * (v1 * np.cos(theta) + v2 * np.sin(theta))).T

In [3]:
rootdir = "/home/kmuvcl/dataset/data/scannet-vp/"
split = 'scannetv2_train.txt'

In [4]:
file_path = rootdir + split
file_path

'/home/kmuvcl/dataset/data/scannet-vp/scannetv2_train.txt'

In [22]:
dirs = np.genfromtxt(file_path, dtype=str)
filelist = sum([sorted(glob.glob(f"{rootdir}{d}/*.png")) for d in dirs], [])
print("total number of samples", len(filelist))

total number of samples 189907


In [ ]:
# dirs = np.genfromtxt(f"{rootdir}/scannetv2_{split}.txt", dtype=str)
# filelist = sum([sorted(glob.glob(f"{rootdir}/{d}/*.png")) for d in dirs], [])
# print("total number of samples", len(filelist))

In [27]:
if split == "scannetv2_train.txt":
    num_train = int(len(filelist) * 1.0)
    print(num_train)
    filelist = filelist[0 : num_train]
    size = len(filelist)
    print(size)

xyz = gold_spiral_sampling_patch(np.array([0, 0, 1]), alpha=90.0 * np.pi / 180., num_pts=16384)
    

189907
189907


In [ ]:
xyz

In [28]:
iname = filelist[0 % len(filelist)]
image = skimage.io.imread(iname)[:, :, 0:3]
print(image.shape)
image = np.rollaxis(image, 2).copy().astype(float)
print(image.shape)

(512, 512, 3)
(3, 512, 512)


In [29]:
with np.load(iname.replace("color.png", "vanish.npz")) as npz:
    vpts = np.array([npz[d] for d in ["x", "y", "z"]])

vpts[:, 1] *= -1
vpts /= LA.norm(vpts, axis=1, keepdims=True)

In [35]:
vpts

array([[-0.99024588, -0.11863921,  0.07306047],
       [ 0.01114918,  0.45520896,  0.89031483],
       [ 0.1388834 , -0.88244512,  0.44944633]])

In [33]:
vpts[2]

array([ 0.1388834 , -0.88244512,  0.44944633])

In [ ]:
dist_cos_arc = cos_cdis(vpts, xyz, semi_sphere=True)

In [ ]:
dist_cos_arc

In [ ]:
vpts_idx = dist_cos_arc.argmin(axis=1)

In [ ]:
vpts_idx

In [ ]:
label = np.zeros((16384), dtype=np.float32)

In [ ]:
label

In [ ]:
label[vpts_idx] = 1.0

In [ ]:
vpts

In [13]:
line_path = "/home/kmuvcl/dataset/data/processed_data/lines/P1020171.txt"

In [19]:
import csv
lsd_line_segments = []

In [20]:
with open(line_path, 'r') as csv_file:
    reader = csv.DictReader(csv_file, delimiter=' ')
    for line in reader:
        p1x = float(line['point1_x'])
        p1y = float(line['point1_y'])
        p2x = float(line['point2_x'])
        p2y = float(line['point2_y'])
        lsd_line_segments += [np.array([p1x, p1y, p2x, p2y])]

In [23]:
line

{'line_hom_a': '0.831623549538148',
 'line_hom_b': '-0.5553397805430218',
 'line_hom_c': '-173.77685439425682',
 'centroid_x': '240.47152981014864',
 'centroid_y': '47.18720629684376',
 'centroid_z': '1.0',
 'point1_x': '244.2258525517585',
 'point1_y': '52.80931966577339',
 'point1_z': '1.0',
 'point2_x': '236.7172070685388',
 'point2_y': '41.56509292791414',
 'point2_z': '1.0'}

In [6]:
rootdir = "/home/kmuvcl/dataset/data/processed_data"

In [138]:
filelist = glob.glob(f"{rootdir}/*_0.png")

In [139]:
filelist.sort()

In [140]:
filelist

['/home/kmuvcl/dataset/data/processed_data/0000_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0001_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0002_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0003_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0004_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0005_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0006_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0007_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0008_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0009_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0010_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0011_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0012_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0013_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0014_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0015_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0016_0.png',
 '/home/kmuvcl/dataset/data/processed_data/0017_

In [24]:
from scipy import io

In [148]:

gt_data = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/vps/P1020171GroundTruthVP_CamParams.mat"))

In [158]:
camera_params = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB", "cameraParameters.mat"))
f = camera_params['focal'][0, 0]
ps = camera_params['pixelSize'][0, 0]
pp = camera_params['pp'][0, :]
K = np.matrix([[f / ps, 0, pp[0]], [0, f / ps, pp[1]], [0, 0, 1]])

In [161]:
camera_params, K

({'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Oct 13 23:06:17 2008',
  '__version__': '1.0',
  '__globals__': [],
  'focal': array([[6.05317059]]),
  'pp': array([[307.55130528, 251.45424496]]),
  'pixelSize': array([[0.00896875]])},
 matrix([[674.91797516,   0.        , 307.55130528],
         [  0.        , 674.91797516, 251.45424496],
         [  0.        ,   0.        ,   1.        ]]))

In [162]:
gt_data['vp']

array([[-0.76923989, -0.06964852,  0.63526196, -8.32560606],
       [-0.15739971,  0.98406444, -0.08427292, -0.47939755],
       [ 0.61926999,  0.16360399,  0.76768504,  1.        ]])

In [150]:
true_vds = np.matrix(gt_data['vp'])

In [178]:
true_vds

matrix([[-0.76923989, -0.06964852,  0.63526196, -8.32560606],
        [ 0.15739971, -0.98406444,  0.08427292,  0.47939755],
        [ 0.61926999,  0.16360399,  0.76768504,  1.        ]])

In [152]:
true_vds[1, :] *= -1

In [165]:
true_vps = K * true_vds

In [167]:
num_vp = true_vps.shape[1]

In [168]:
tvp_list = []
for vi in range(num_vp):
    true_vps[:, vi] /= true_vps[2, vi]

    tVP = np.array(true_vps[:, vi])[:, 0]
    tVP /= tVP[2]

    tvp_list += [tVP]

In [169]:
true_vps = np.vstack(tvp_list)

In [172]:
true_vps = torch.tensor(true_vps)

In [184]:
vp = true_vps

In [185]:
vp

tensor([[-5.3081e+02,  4.2300e+02,  1.0000e+00],
        [ 2.0229e+01, -3.8081e+03,  1.0000e+00],
        [ 8.6605e+02,  3.2554e+02,  1.0000e+00],
        [-5.3115e+03,  5.7501e+02,  1.0000e+00]], dtype=torch.float64)

In [189]:
vps_pixel = vp[:, 0:2]
vps_pixel

tensor([[ -530.8130,   422.9980],
        [   20.2292, -3808.1215],
        [  866.0482,   325.5436],
        [-5311.5499,   575.0083]], dtype=torch.float64)

In [201]:
vps_homo = np.concatenate((vps_pixel, np.ones((len(vps_pixel),1), dtype=np.float32)), axis=-1)

In [202]:
vps_homo

array([[-5.30812950e+02,  4.22997993e+02,  1.00000000e+00],
       [ 2.02292286e+01, -3.80812146e+03,  1.00000000e+00],
       [ 8.66048212e+02,  3.25543624e+02,  1.00000000e+00],
       [-5.31154988e+03,  5.75008266e+02,  1.00000000e+00]])

In [203]:
vps_homo[:, 0] -= 320
vps_homo[:, 1] -= 240
vps_homo[:, 1] *= -1

In [204]:
vps_homo

array([[-8.50812950e+02, -1.82997993e+02,  1.00000000e+00],
       [-2.99770771e+02,  4.04812146e+03,  1.00000000e+00],
       [ 5.46048212e+02, -8.55436241e+01,  1.00000000e+00],
       [-5.63154988e+03, -3.35008266e+02,  1.00000000e+00]])

In [205]:
vps_homo[:, 0:2] /= 320.

In [177]:
torch.dot(true_vps[0],true_vps[1])

tensor(-1621564.6715, dtype=torch.float64)

In [111]:
iname = "/home/kmuvcl/dataset/data/processed_data/0000_0.png"

In [112]:
image = skimage.io.imread(iname)[:, :, 0:3]

In [113]:
image.shape

(480, 640, 3)

In [179]:
with np.load(iname.replace(".png", ".npz"), allow_pickle=True) as npz:
    vpts = npz["vpts"]
    dict = npz['line_segments']
    vp_inds = npz['line_vp_inds']
    # vpts = vpts[0:3]

In [180]:
vpts

array([[-0.9175774 , -0.1973581 ,  0.34511084],
       [-0.0736212 ,  0.99418491,  0.07858933],
       [ 0.85499071, -0.13394239,  0.50104922],
       [-0.99663327, -0.05928748,  0.05663142]])

In [131]:
elements , counts = np.unique(vp_inds,return_counts = True)
print(elements,counts)

[0 1 2 3] [ 54 425 244  63]


In [132]:
sorted_indices = np.argsort(-counts)
sorted_indices

array([1, 2, 3, 0])

In [134]:
top_3_indices = elements[sorted_indices[:3]]
top_3_indices

array([1, 2, 3])

In [135]:
top_3_vpts = vpts[top_3_indices]

In [136]:
top_3_vpts

array([[-0.0736212 ,  0.99418491,  0.07858933],
       [ 0.85499071, -0.13394239,  0.50104922],
       [-0.99663327, -0.05928748,  0.05663142]])

In [137]:
vpts 

array([[-0.9175774 , -0.1973581 ,  0.34511084],
       [-0.0736212 ,  0.99418491,  0.07858933],
       [ 0.85499071, -0.13394239,  0.50104922],
       [-0.99663327, -0.05928748,  0.05663142]])

In [18]:
camera_params = io.loadmat(os.path.join("/home/kmuvcl/dataset/data/processed_data/YorkUrbanDB/P1020171", "P1020171GroundTruthVP_Orthogonal_CamParams.mat"))

In [21]:
gt_ortho_data = camera_params['vp_orthogonal']

In [22]:
gt_ortho_data

array([[ 0.76923989, -0.06924798,  0.63519659],
       [ 0.15739971,  0.98401208, -0.08334001],
       [-0.61926999,  0.16408822,  0.76784095]])

In [23]:
gt_ortho_data[1,:]

array([ 0.15739971,  0.98401208, -0.08334001])